In [1]:
#import pandas as pd
import numpy as np
from multiprocessing import Process, Pipe
from sklearn.model_selection import train_test_split
from os import cpu_count

In [2]:
try:
    with open('./input/inputs.npy',"rb") as f:
        inputs = np.load(f)
    with open('./input/outputs.npy',"rb") as f:
        outputs = np.load(f)
        
except:
    data = np.genfromtxt('./input/train.csv', delimiter=',',skip_header=1,dtype=np.uint8)
    inputs = data[:,1:]
    outputs = data[:,0]
    np.save('./input/inputs.npy',inputs)
    np.save('./input/outputs.npy',outputs)


print(f'Input:\n{inputs[:5,:5]}\n')
print(f'Output:\n{outputs}\n')

Input:
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Output:
[1 0 1 ... 7 6 9]



In [14]:
testsize = 0.99

class NeuralNetwork():
    
    def __init__(self):
        # seeding for random number generation
        np.random.seed(1)
        
        #converting weights to values from -1 to 1 and mean of 0
        #self.weights = (2 * np.random.random((5, 3)) - 1).astype(np.float16)
        self.weights = (2 * np.random.random((round(42000*(1-testsize)),784)) - 1).astype(np.float16)

    def sigmoid(self, x):
        #applying the sigmoid function
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        #computing derivative to the Sigmoid function
        #print('sigmoid')
        return x * (1 - x)

    def train(self, training_inputs, training_outputs, training_iterations, bot, top, pipe):
        #training the model to make accurate predictions while adjusting weights continually
        for iteration in range(training_iterations):
            #siphon the training data via  the neuron
            output = self.feedforward(training_inputs,bot,top)

            #computing error rate for back-propagation
            print(f'train {training_outputs.shape}')
            print(f'outputs {output.shape}')
            error = training_outputs - output.T

            #performing weight adjustments
            adjustments = np.dot(training_inputs[bot:top,:], error.T * self.sigmoid_derivative(output))
            self.weights += adjustments
            
            #phone home
        pipe[1].send(self.weights)
        pipe[1].close()
        
    def feedforward(self, inputs, bot, top):
        output = self.sigmoid(np.dot(inputs[bot:top,:], self.weights[bot:top,:]))
        print(f'outputshape{output.shape}')
        return output
    
    def communicate(self, conn, data):
        conn.send(data)       

In [15]:
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size=testsize)

if __name__ == "__main__":

    #initializing the neuron class
    network = NeuralNetwork()

    print("Beginning Randomly Generated Weights: ")
    print(network.weights[0:5])

    #training data consisting of 4 examples--3 input values and 1 output
    training_inputs = X_train# np.random.random((3,5))#
    training_outputs = y_train.T # np.array([1,2,3])#
    iterations = 1

Beginning Randomly Generated Weights: 
[[-0.166   0.4407 -1.     ...  0.9507 -0.3594  0.965 ]
 [ 0.2727 -0.2499  0.715  ... -0.643   0.94    0.484 ]
 [-0.4805  0.1625  0.9106 ...  0.3965 -0.7065  0.225 ]
 [-0.4414  0.8315 -0.6035 ... -0.873   0.1686 -0.3286]
 [ 0.7637  0.7476  0.975  ...  0.1312  0.281   0.5356]]


In [16]:
network = NeuralNetwork()
network.feedforward(np.random.random((420,784)),0,105)


print(training_inputs[0:5])
print(training_inputs.shape)
network.weights.shape

ValueError: shapes (105,784) and (105,784) not aligned: 784 (dim 1) != 105 (dim 0)

In [17]:
NeuralNetwork.feedforward(NeuralNetwork(),np.random.random(420),0,420)
print(training_inputs[0:5])
print(training_inputs.shape)
network.weights.shape

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size=testsize)

if __name__ == "__main__":

    #initializing the neuron class
    network = NeuralNetwork()

    print("Beginning Randomly Generated Weights: ")
    print(network.weights[0:5])

    #training data consisting of 4 examples--3 input values and 1 output
    training_inputs = X_train# np.random.random((3,5))#
    training_outputs = y_train.T # np.array([1,2,3])#
    iterations = 1

    jobs= []
    procs= 4#cpu_count()
    pipes = []
    #training taking place
    for i in range(0, procs):
        pipes.append(Pipe()) # [Parent,Child]
        bot = i*int(training_inputs.shape[0]/procs)
        top = (i+1)*int(training_inputs.shape[0]/procs)
        print(bot)
        print(training_inputs[bot:top,:].shape)
        
        print(top)
        print(training_outputs[bot:top].shape)
        process = Process(target=network.train,
                          args=(training_inputs,
                                training_outputs,
                                bot,
                                top,
                                iterations,
                                pipes[i]))
        jobs.append(process)
    
    
    # Start the processes
    for j in jobs:
        j.start()
        print(f' Go process {j.pid}!')

    # Ensure all of the processes have finished
    i = 0
    for j in jobs:
        bot = i*int(training_inputs.shape[0]/procs)
        top = (i+1)*int(training_inputs.shape[0]/procs)
        out = pipes[i][0].recv()
        print(out)
        #network.weights[:,bot:top] = out
        i = i + 1
        j.join()


    print("Ending Weights After Training: ")
    print(network.weights[0:5])

Beginning Randomly Generated Weights: 
[[-0.166   0.4407 -1.     ...  0.9507 -0.3594  0.965 ]
 [ 0.2727 -0.2499  0.715  ... -0.643   0.94    0.484 ]
 [-0.4805  0.1625  0.9106 ...  0.3965 -0.7065  0.225 ]
 [-0.4414  0.8315 -0.6035 ... -0.873   0.1686 -0.3286]
 [ 0.7637  0.7476  0.975  ...  0.1312  0.281   0.5356]]
0
(105, 784)
105
(105,)
105
(105, 784)
210
(105,)
210
(105, 784)
315
(105,)
315
(105, 784)
420
(105,)
 Go process 18785!
 Go process 20220!
outputshape[] Go process 16421!

train (420,)
outputs (0, 0)
outputshape[]

Process Process-2:


outputshape[]
train (420,)

Traceback (most recent call last):



outputs (0, 0)

...

ValueError: operands could not be broadcast together with shapes (420,) (0,0) 
Traceback (most recent call last):



outputs (0, 0)

  File "/usr/pkg/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()


  File "/usr/pkg/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-3:
  File "<ipython-input-3-366873a3bf23>", line 31, in train
    error = training_outputs - output.T
ValueError: operands could not be broadcast together with shapes (420,) (0,0) 
Traceback (most recent call last):
  File "/usr/pkg/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/pkg/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-366873a3bf23>", line 31, in train
    error = training_outputs - output.T
ValueError: operands could not be broadcast together with shapes (420,) (0,0) 


 Go process 20521!
[[-0.166    0.4407  -1.      ...  0.9507  -0.3594   0.965  ]
 [ 0.2727  -0.2499   0.715   ... -0.643    0.94     0.484  ]
 [-0.4805   0.1625   0.9106  ...  0.3965  -0.7065   0.225  ]
 ...
 [-0.8867  -0.4854   0.749   ... -0.1747   0.887   -0.756  ]
 [ 0.3054  -0.55     0.7065  ...  0.5493  -0.755    0.577  ]
 [-0.5205   0.2505   0.3389  ... -0.2576  -0.00979  0.4426 ]]


In [6]:
import numpy as np
a = np.array([[1,1,1],[2,2,2],[3,3,3],[4,4,4]])
b = np.array([-1,-1,-1])
print(a.shape)
print(b.shape)
a + b

(4, 3)
(3,)


array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])